In [ ]:
import pandas as pd
import sklearn
import numpy as np


#### Cleaning the Data file - Dropping Features and Converting Categorical to Numerical


In [ ]:
data = pd.read_csv("data/out.csv")

# Modify the 'winner' column based on conditions
data['winner'] = data.apply(lambda row: 1 if row['winner'] == row['fighter_1'] else 0, axis=1)


# Drop the specified columns
columns_to_drop = ['fight_id', 'fighter_1_id', 'fighter_1', 'fighter_2', 
'fighter_2_id','decision_method','fight_duration_lastrnd','time_format','date', 
'location','height_x','name_x','weight_x','height_y','name_y','weight_y' ]

data_cleaned = data.drop(columns=columns_to_drop)

# Display the first few rows of the cleaned data
data_cleaned.head()

data_cleaned.to_csv("data/cleanedData2.csv", index = False)

In [ ]:
import pandas as pd
import sklearn
import numpy as np


#### Cleaning the Data file - Dropping Features and Converting Categorical to Numerical


In [ ]:
data = pd.read_csv("data/out.csv")

# Modify the 'winner' column based on conditions
data['winner'] = data.apply(lambda row: 1 if row['winner'] == row['fighter_1'] else 0, axis=1)


# Drop the specified columns
columns_to_drop = ['fight_id', 'fighter_1_id', 'fighter_1', 'fighter_2', 
'fighter_2_id','decision_method','fight_duration_lastrnd','time_format','date', 
'location','height_x','name_x','weight_x','height_y','name_y','weight_y' ]

data_cleaned = data.drop(columns=columns_to_drop)

# Display the first few rows of the cleaned data
data_cleaned.head()

data_cleaned.to_csv("data/cleanedData2.csv", index = False)

In [ ]:
df = pd.read_csv("data/cleanedDataNEW_encoded.csv") # original dataset
##print(f"Initial shape: {df.shape}")



df

,winner,decision_method,fight_duration_lastrnd,fight_duration_lastrnd_time,time_format,weight_class,date,location,dob_x,height_x,...,win_percentage_x,win_percentage_y,reach_advantage,total_fights_x,total_fights_y,finishing_rate_x,finishing_rate_y,age_x,age_y,age_difference
0,0,3,3,292,1,2,"November 02, 2019",92,835,"5' 7""",...,0.800000,0.846154,-3.0,20.0,13.0,0.242500,0.406154,30,28,2
1,1,7,3,164,1,0,"September 28, 2019",37,256,"5' 8""",...,1.000000,0.714286,0.0,12.0,14.0,0.398333,0.094286,24,25,-1
2,0,4,3,292,1,58,"November 16, 2019",126,612,"5' 5""",...,0.588235,0.875000,0.0,17.0,8.0,0.211765,0.603750,31,25,6
3,1,7,3,173,1,0,"October 05, 2019",78,286,"5' 5""",...,0.875000,0.687500,4.0,16.0,16.0,0.196250,0.096250,27,29,-2
4,0,4,3,292,1,4,"October 18, 2019",23,466,"6' 7""",...,0.666667,0.739130,7.0,9.0,23.0,0.288889,0.269565,29,28,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3822,1,4,3,292,1,3,"September 21, 2019",80,126,"5' 6""",...,0.782609,0.620690,1.0,23.0,29.0,0.186087,0.092069,26,35,-9
3823,0,8,3,109,1,61,"September 21, 2019",80,737,"5' 2""",...,0.857143,0.588235,-3.0,14.0,17.0,0.430714,0.351765,26,34,-8
3824,1,4,3,292,1,58,"September 21, 2019",80,606,"5' 9""",...,0.687500,0.588235,3.0,16.0,17.0,0.393750,0.211765,31,31,0
3825,0,5,2,84,1,2,"September 21, 2019",80,901,"5' 8""",...,0.800000,0.666667,1.0,15.0,27.0,0.510667,0.187037,26,29,-3


In [ ]:
##shuffling the data 
df_shuffled = df.sample(frac = 1)
df_shuffled.head(n=5)



In [ ]:
##split up the data 
df_shuffled.columns

'''x = df_shuffled.drop([], axis = 1).values
y = df_shuffled["Winner"].values'''
y = df_shuffled.iloc[:,0]
x = df_shuffled.iloc[:,1]

### Making min max scaler

In [ ]:
import numpy as np

class CustomMinMaxScaler:
    def __init__(self):
        self.min_ = None
        self.max_ = None

    def fit(self, data):
        # Compute min and max values for each feature
        self.min_ = np.min(data, axis=0)
        self.max_ = np.max(data, axis=0)

    def transform(self, data):
        # Scale data to the range [0, 1]
        return (data - self.min_) / (self.max_ - self.min_)

    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)

### Splitting up the data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
xTrain, xTest, yTrain, yTest = train_test_split(x,y,test_size = .2)

In [ ]:
'''winners = y = df_shuffled["Winner"].values'''

scaler = CustomMinMaxScaler()

xTrainScaled = scaler.fit_transform(xTrain)
xTestScaled = scaler.fit_transform(xTest)
yTrainScaled = scaler.fit_transform(yTrain)
yTestScaled = scaler.fit_transform(yTest)


### Encoding Data

In [ ]:
import pandas as pd

# Load the dataset
file_path = 'data/cleanedDataNEW.csv'
data = pd.read_csv(file_path)

# Specify the columns to encode
columns_to_encode = ['decision_method', 'time_format', 'weight_class']

# Perform one-hot encoding
encoded_data = pd.get_dummies(data, columns=columns_to_encode, drop_first=True)

# Save the modified dataset
output_file_path = 'data/NEWencoded_cleanedDataNEW.csv'
encoded_data.to_csv(output_file_path, index=False)

print("done")


### Training the model

In [ ]:

import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

# Function to convert height strings to numeric (in inches)
def convert_height(height):
    try:
        parts = height.split("'")
        feet = int(parts[0])
        inches = int(parts[1].replace('"', '').strip())
        return feet * 12 + inches
    except:
        return np.nan  # Return NaN if conversion fails

# Load the dataset
file_path = "data/cleanedDataNEW_encoded.csv"  
data = pd.read_csv(file_path)

# Convert height columns to numeric
if 'height_x' in data.columns:
    data['height_x'] = data['height_x'].apply(convert_height)
if 'height_y' in data.columns:
    data['height_y'] = data['height_y'].apply(convert_height)

# Fill missing height values with the median height
for col in ['height_x', 'height_y']:
    if col in data.columns:
        data[col].fillna(data[col].median(), inplace=True)


# Define target column
target = 'winner'

# Handle categorical data
categorical_columns = ['decision_method', 'stance_x', 'stance_y', 'time_format', 'weight_class']
available_categorical = [col for col in categorical_columns if col in data.columns]
data_encoded = pd.get_dummies(data, columns=available_categorical, drop_first=True)

# Define the columns to exclude
exclude_columns = [
    'weight_x', 'weight_y', 'fight_duration_lastrnd_time', 'date', 'location',
    'dob_x', 'name_x', 'dob_y', 'name_y', target  
]

# Prepare features for training by excluding specific columns
existing_features = [col for col in data_encoded.columns if col not in exclude_columns]
print("Features used for training:", existing_features)

transformed_file_path = "data/transformedData.csv"
data_encoded.to_csv(transformed_file_path, index=False)
print(f"saved to {transformed_file_path}")
# Prepare data for training
X = data_encoded[existing_features]
y = data_encoded[target]

# Check for missing values and handle them
if X.isnull().sum().sum() > 0:
    print("imputing missing vals")
    X.fillna(X.median(), inplace=True)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Gradient Boosting model
model = GradientBoostingClassifier(
    n_estimators=100,       # Number of trees
    learning_rate=0.1,      # Learning rate
    max_depth=3,            # Maximum tree depth
    min_samples_split=2,    # Stop splitting nodes with maximum instances
    subsample=1.0,          # Fraction of training instances
    random_state=42         # Replicable training
)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
# print("\nClassification Report:\n", classification_report(y_test, y_pred))
# print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Save the trained model to a file
joblib_file = "joblibs/gradient_boosting_model.joblib"  # Specify the file name
joblib.dump(model, joblib_file)

print(f"Model saved to {joblib_file}")



In [ ]:
import numpy as np
import joblib 
import sklearn

print("Numpy version:", np.__version__)
print("Joblib version:", joblib.__version__)
print("scikit-learn", sklearn.__version__)


### Loading in the Model to make predictions

In [ ]:
import joblib
import numpy as np
import pandas as pd

# Load the model
testingModel = joblib.load("joblibs/gradient_boosting_model.joblib")

# Path to the input file (replace with your actual file path)
file_path = "data/transformedData.csv"

# Read the file into a DataFrame
data = pd.read_csv(file_path)

# Specify the row index and the desired columns (replace with your selections)
row_index = 3  # Select the row you want (e.g., 0 for the first row)
desired_columns = np.array(['fight_duration_lastrnd', 'height_x', 'n_draw_x', 'n_loss_x', 'n_win_x', 'reach_x', 'sig_str_abs_pM_x', 'sig_str_def_pct_x', 'sig_str_land_pM_x', 'sig_str_land_pct_x', 'sub_avg_x', 'td_avg_x', 'td_def_pct_x', 'td_land_pct_x', 'height_y', 'n_draw_y', 'n_loss_y', 'n_win_y', 'reach_y', 'sig_str_abs_pM_y', 'sig_str_def_pct_y', 'sig_str_land_pM_y', 'sig_str_land_pct_y', 'sub_avg_y', 'td_avg_y', 'td_def_pct_y', 'td_land_pct_y', 'win_percentage_x', 'win_percentage_y', 'reach_advantage', 'total_fights_x', 'total_fights_y', 'finishing_rate_x', 'finishing_rate_y', 'age_x', 'age_y', 'age_difference', 'decision_method_1', 'decision_method_2', 'decision_method_3', 'decision_method_4', 'decision_method_5', 'decision_method_6', 'decision_method_7', 'decision_method_8', 'stance_x_1', 'stance_x_2', 'stance_x_3', 'stance_y_1', 'stance_y_2', 'stance_y_3', 'time_format_1', 'time_format_2', 'time_format_3', 'weight_class_1', 'weight_class_2', 'weight_class_3', 'weight_class_4', 'weight_class_5', 'weight_class_6', 'weight_class_7', 'weight_class_8', 'weight_class_9', 'weight_class_10', 'weight_class_11', 'weight_class_12', 'weight_class_13', 'weight_class_14', 'weight_class_15', 'weight_class_16', 'weight_class_17', 'weight_class_18', 'weight_class_19', 'weight_class_20', 'weight_class_21', 'weight_class_22', 'weight_class_23', 'weight_class_24', 'weight_class_25', 'weight_class_26', 'weight_class_27', 'weight_class_28', 'weight_class_29', 'weight_class_30', 'weight_class_31', 'weight_class_32', 'weight_class_33', 'weight_class_34', 'weight_class_35', 'weight_class_36', 'weight_class_37', 'weight_class_38', 'weight_class_39', 'weight_class_40', 'weight_class_41', 'weight_class_42', 'weight_class_43', 'weight_class_44', 'weight_class_45', 'weight_class_46', 'weight_class_47', 'weight_class_48', 'weight_class_49', 'weight_class_50', 'weight_class_51', 'weight_class_52', 'weight_class_53', 'weight_class_54', 'weight_class_55', 'weight_class_56', 'weight_class_57', 'weight_class_58', 'weight_class_59', 'weight_class_60', 'weight_class_61'])

# Extract the selected row and columns
selected_row = data.loc[row_index, desired_columns]

# Convert the selected row into a NumPy array
input_features = np.array(selected_row).astype(float)  # Ensure all values are numerical

# Reshape the input for the model
reshaped_input = input_features.reshape(1, -1)

# Make predictions
predictions = testingModel.predict(reshaped_input)

print("Prediction:", predictions)
print("done")



In [ ]:
"""
Script for Enhancing UFC Fighters Dataset

This script enhances the "ufc_fighters_stats.csv" dataset by adding derived features:
1. Calculates the age of each fighter based on their date of birth (DOB) and today's date.
2. Converts wins, losses, and draws to numeric values, handling non-numeric entries gracefully.
3. Computes total fights as the sum of wins, losses, and draws.
4. Calculates the win percentage for each fighter.
5. Removes unnecessary columns (e.g., 'url').

Usage:
- Ensure the "ufc_fighters_stats.csv" file is present in the same directory as this script.
- The DOB format in the dataset should be "Sep20,1989".
- The enhanced dataset will be saved as "ufc_fighters_statsnew.csv".

Dependencies:
- pandas: For data manipulation
- datetime: For calculating fighter age
"""

import pandas as pd
from datetime import datetime

file_path = 'data/ufc_fighters_stats.csv'
data = pd.read_csv(file_path)


# Function to calculate age as of today
def calculate_age_as_of_today(dob):
    if pd.isna(dob) or dob.strip() == "":
        return None
    try:
        # Adjusted format for "Sep20,1989"
        dob_date = datetime.strptime(dob.strip(), '%b%d,%Y')
        today = datetime.today()
        # Calculate age
        age = today.year - dob_date.year - ((today.month, today.day) < (dob_date.month, dob_date.day))
        return age
    except ValueError as e:
        return None


# Convert wins, losses, and draws to numeric 
data['wins'] = pd.to_numeric(data['wins'], errors='coerce')
data['losses'] = pd.to_numeric(data['losses'], errors='coerce')
data['draws'] = pd.to_numeric(data['draws'], errors='coerce')

# Fill missing values with 0 
data['wins'].fillna(0, inplace=True)
data['losses'].fillna(0, inplace=True)
data['draws'].fillna(0, inplace=True)

# Calculate total fights
data['total_fights'] = data['wins'] + data['losses'] + data['draws']

# Calculate age based on 'dob'
data['age'] = data['dob'].apply(calculate_age_as_of_today)

data['win_percentage'] = data['wins'] / (data['wins'] + data['losses'] + data['draws'])

data = data.drop(columns=['url'])

# Save the updated dataset
data.to_csv("ufc_fighters_statsnew.csv", index=False)

In [ ]:
"""
Script for UFC Fight Winner Prediction

This script uses a pre-trained Gradient Boosting model to predict the winner between two UFC fighters 
based on their statistical features. It includes the following steps:
1. Load a dataset containing UFC fighter statistics.
2. Preprocess the data to handle numeric and categorical features.
3. Dynamically compute features for prediction based on two input fighter names.
4. Predict the winner using the trained model.

Key Assumptions:
- The model was trained with specific feature names and expects inputs in a consistent format.
- The dataset ("ufc_fighters_statsnew.csv") contains fighter statistics such as height, reach, 
  wins, losses, draws, significant strikes, takedown averages, and more.

Usage:
- Replace `fighter1` and `fighter2` with the names of two fighters in the dataset.

Dependencies:
- pandas: For data manipulation
- joblib: For loading the pre-trained model
"""

import pandas as pd
import joblib

# Load the trained model
model = joblib.load("joblibs/gradient_boosting_model.joblib")

# Load fighter stats dataset
fighter_stats = pd.read_csv("ufc_fighters_statsnew.csv")

print(fighter_stats.columns)


# Ensure numeric conversion for relevant columns
numeric_columns = [
    "height", "draws", "losses", "wins", "reach", "sapm", "age",
    "str_def", "slpm", "str_acc", "sub_avg", 
    "td_avg", "td_def", "td_acc", "win_percentage", "total_fights"
]
for col in numeric_columns:
    if col in fighter_stats.columns:
        fighter_stats[col] = pd.to_numeric(fighter_stats[col], errors='coerce')

# Fill missing numeric values with 0
fighter_stats[numeric_columns] = fighter_stats[numeric_columns].fillna(0)

def get_fighter_stats(name, data):
    """
    Retrieve fighter stats by name from the dataset.
    """
    return data[data['name'] == name]

def prepare_features(fighter1, fighter2, data):
    """
    Prepare features for prediction based on two fighter names.
    """
    # Retrieve stats for both fighters
    stats_x = get_fighter_stats(fighter1, data)
    stats_y = get_fighter_stats(fighter2, data)
    
    if stats_x.empty or stats_y.empty:
        raise ValueError("One or both fighter names not found in the dataset.")
    
    # Compute derived features
    features = {
        "decision_method": 1, 
        "fight_duration_lastrnd": 3,  # Default round duration
        "time_format": 5,  # Default time format
        
        # Fighter X stats
        "weight_class": 1,  # Default weight class encoding
        "height_x": stats_x["height"].values[0],
        "n_draw_x": stats_x["draws"].values[0],
        "n_loss_x": stats_x["losses"].values[0],
        "n_win_x": stats_x["wins"].values[0],
        "reach_x": stats_x["reach"].values[0],
        "sig_str_abs_pM_x": stats_x["sapm"].values[0],
        "sig_str_def_pct_x": stats_x["str_def"].values[0],
        "sig_str_land_pM_x": stats_x["slpm"].values[0],
        "sig_str_land_pct_x": stats_x["str_acc"].values[0],
        "stance_x": stats_x["stance"].values[0],
        "sub_avg_x": stats_x["sub_avg"].values[0],
        'td_avg_x': stats_x["td_avg"].values[0],  
        'td_def_pct_x': stats_x["td_def"].values[0],  
        'td_land_pct_x': stats_x["td_acc"].values[0], 
        "win_percentage_x": stats_x["win_percentage"].values[0],
        "total_fights_x": stats_x["total_fights"].values[0],
        "age_x": stats_x["age"].values[0],
        
        # Fighter Y stats
        "weight_class": 1,  # Default weight class encoding
        "height_y": stats_x["height"].values[0],
        "n_draw_y": stats_x["draws"].values[0],
        "n_loss_y": stats_x["losses"].values[0],
        "n_win_y": stats_x["wins"].values[0],
        "reach_y": stats_x["reach"].values[0],
        "sig_str_abs_pM_y": stats_x["sapm"].values[0],
        "sig_str_def_pct_y": stats_x["str_def"].values[0],
        "sig_str_land_pM_y": stats_x["slpm"].values[0],
        "sig_str_land_pct_y": stats_x["str_acc"].values[0],
        "stance_y": stats_x["stance"].values[0],
        "sub_avg_y": stats_x["sub_avg"].values[0],
        'td_avg_y': stats_x["td_avg"].values[0],  
        'td_def_pct_y': stats_x["td_def"].values[0], 
        'td_land_pct_y': stats_x["td_acc"].values[0],

        "win_percentage_y": stats_x["win_percentage"].values[0],
        "total_fights_y": stats_x["total_fights"].values[0],
        "age_y": stats_x["age"].values[0],
        
        # Derived features
        "reach_advantage": stats_x["reach"].values[0] - stats_y["reach"].values[0],
        "age_difference": stats_x["age"].values[0] - stats_y["age"].values[0],
    }
    
    # Ensure feature alignment with the model
    features_df = pd.DataFrame([features])
    for col in model.feature_names_in_:
        if col not in features_df.columns:
            features_df[col] = 0  # Add missing features with default value
    features_df = features_df[model.feature_names_in_]  # Align column order
    
    return features_df

# Example usage
# open "ufc_fighters_statsnew.csv" and select which fighter to use for prediction
fighter1 = "Shavkat Rakhmonov"  # example
fighter2 = "Ian Garry"  # example

try:
    # Prepare features for prediction
    input_features = prepare_features(fighter1, fighter2, fighter_stats)
    print("Prepared features for prediction:\n", input_features)
    
    # Use the model for prediction
    prediction = model.predict(input_features)
    if prediction[0] == 0:
        winner = fighter1
    else:
        winner = fighter2
    print(f"The predicted winner between {fighter1} and {fighter2} is: {winner}")

except ValueError as e:
    print(e)

Index(['name', 'wins', 'losses', 'draws', 'height', 'weight', 'reach',
       'stance', 'dob', 'slpm', 'str_acc', 'sapm', 'str_def', 'td_avg',
       'td_acc', 'td_def', 'sub_avg', 'last_update', 'last_match',
       'total_fights', 'age', 'win_percentage'],
      dtype='object')
Prepared features for prediction:
    fight_duration_lastrnd  height_x  n_draw_x  n_loss_x  n_win_x  reach_x  \
0                       3       0.0       0.0         0       17      0.0   

   sig_str_abs_pM_x  sig_str_def_pct_x  sig_str_land_pM_x  sig_str_land_pct_x  \
0              2.61                0.0               4.45                 0.0   

   ...  weight_class_52  weight_class_53  weight_class_54  weight_class_55  \
0  ...                0                0                0                0   

   weight_class_56  weight_class_57  weight_class_58  weight_class_59  \
0                0                0                0                0   

   weight_class_60  weight_class_61  
0                0      